In [8]:
import numpy as np
from scipy.stats import vonmises_fisher, uniform_direction
import itertools

In [9]:
class background_functions:

    def __init__(self, dimension, radius):

        self.dimension = dimension
        self.radius = radius


    #Generation of random coefficients model samples
    def random_coefficients_generation(self, coefficients_type, sample_size):

        covariate_mean = np.ones(self.dimension).reshape(self.dimension) / np.sqrt(self.dimension)
        covariate_samples = vonmises_fisher.rvs(mu = covariate_mean, kappa = 0.1, size = sample_size).T

        if coefficients_type == "spherical":
            coefficients_samples = uniform_direction.rvs(dim = self.dimension, size = sample_size).T  * self.radius / 4
            coefficients_samples = coefficients_samples @ (np.diag(np.random.uniform(size = sample_size) ** (1 / self.dimension)))
            for iter in range(sample_size):
              binomial_sample = np.random.binomial(1, 0.5, 1)[0]
              if binomial_sample == 0:
                coefficients_samples[:, iter] += np.eye(1, self.dimension, 1).reshape(self.dimension) * self.radius / 2
              else:
                coefficients_samples[:, iter] -= np.eye(1, self.dimension, 1).reshape(self.dimension) * self.radius / 2

        elif coefficients_type == "degenerate":
            coefficients_samples = uniform_direction.rvs(dim = self.dimension, size = sample_size).T  * self.radius / 4
            for iter in range(sample_size):
              binomial_sample = np.random.binomial(1, 0.5, 1)[0]
              if binomial_sample == 0:
                coefficients_samples[:, iter] += np.eye(1, self.dimension, 1).reshape(self.dimension) * self.radius / 2
              else:
                coefficients_samples[:, iter] -= np.eye(1, self.dimension, 1).reshape(self.dimension) * self.radius / 2

        elif coefficients_type == "discrete":
            mass_sample_size = 2 * self.dimension
            mass_sample = list(np.concatenate((np.identity(self.dimension), -np.identity(self.dimension)), axis = 1).T * self.radius / 2)
            mass_index = np.arange(mass_sample_size)
            mass_weight = np.ones(mass_sample_size) / mass_sample_size
            index_samples = np.random.choice(mass_index, size = sample_size, p = mass_weight)
            coefficients_samples = np.array([mass_sample[index] for index in index_samples]).T

        dependent_samples = np.diag(covariate_samples.T @ coefficients_samples)

        return covariate_samples, coefficients_samples, dependent_samples

In [10]:
for dimension in range(2, 3):
    function = background_functions(dimension, 10)
    for (sample_style, sample_size) in itertools.product(["spherical", "degenerate", "discrete"], [500, 1000, 1500, 2000]):
        covariate_samples, coefficient_samples, dependent_samples = function.random_coefficients_generation(sample_style, sample_size)
        np.save(sample_style + "_covariate_samples_" + str(dimension) + "_" + str(sample_size), covariate_samples)
        np.save(sample_style + "_coefficient_samples_" + str(dimension) + "_" + str(sample_size), coefficient_samples)
        np.save(sample_style + "_dependent_samples_" + str(dimension) + "_" + str(sample_size), dependent_samples)

In [11]:
for dimension in range(3, 6):
    function = background_functions(dimension, 10)
    for (sample_style, sample_size) in itertools.product(["spherical"], [500]):
        covariate_samples, coefficient_samples, dependent_samples = function.random_coefficients_generation(sample_style, sample_size)
        np.save(sample_style + "_covariate_samples_" + str(dimension) + "_" + str(sample_size), covariate_samples)
        np.save(sample_style + "_coefficient_samples_" + str(dimension) + "_" + str(sample_size), coefficient_samples)
        np.save(sample_style + "_dependent_samples_" + str(dimension) + "_" + str(sample_size), dependent_samples)